Dana Rozenblum & Nitzan Tal
# Assignment 3 Solutions


In [2]:
import numpy as np
import scipy.stats

## Model quality

In [9]:
newPositive = 1235
n = 1600
newProb = newPositive / n
h0Prob = 0.75
h0Var = h0Prob * (1 - h0Prob)
means = []
bootSize = 10000

z = (newProb - h0Prob) / np.sqrt(h0Var / n)
pValue = 1 - scipy.stats.norm(0, 1).cdf(z)
pValueP = round(pValue * 100,1)
print ("Analytical p-value: " + str(pValueP) + "%")

# Bootstrapped p value: simulation-like approach, sampling from H0
sample1 =  np.random.binomial(n,h0Prob,bootSize) / n
pval= 1-np.mean(newProb > sample1)
pval = round(pval * 100,1)

print ("Bootstrapped p-value: " + str(pval) + "%")

Analytical p-value: 2.2%
Bootstrapped p-value: 2.2%


Last run result:

Analytical p-value: 2.2%

Bootstrapped p-value: 2.2%

## Comparing salaries

In [ ]:
data1 = np.genfromtxt('salaries1.txt')
data2 = np.genfromtxt('salaries2.txt')
sig = 0.05

mean1 = np.mean(data1)
mean2 = np.mean(data2)
n1 = data1.size
n2 = data2.size
 
s1_2 = scipy.stats.tvar(data1)/n1
 
s2_2 = scipy.stats.tvar(data2)/n2
denominator = np.sqrt(s1_2 + s2_2)
t = (mean1 - mean2) / denominator
df_t = n1 + n2 - 2

p_value = 2 * scipy.stats.t(df = df_t).cdf(t)

pValuePrint = round(p_value * 100,1)
sigPrint = round(sig * 100,0)
print("Two-sided p-value is " + str(pValuePrint) + "%")
print("At " + str(sigPrint) + "% significance:")
if (p_value < sig):
    print ("The null hypothesis that the salary for both occupations is the same is rejected")
else:
    print ("The null hypothesis that the salary for both occupations is the same cannot be rejected")
    
#Sanity check
print("Sanity check: calculate with scipy.stats.ttest_ind:")
print(scipy.stats.ttest_ind(data1,data2,equal_var = False))

Two-sided p-value is 3.5%
At 5.0% significance:
The null hypothesis that the salary for both occupations is the same is rejected
Sanity check: calculate with scipy.stats.ttest_ind:
Ttest_indResult(statistic=-2.129970166236807, pvalue=0.034916573358014256)


Last run result:

Two-sided p-value is 3.5%

At 5.0% significance:
The null hypothesis that the salary for both occupations is the same is rejected.

Sanity check: calculate with scipy.stats.ttest_ind:

Ttest_indResult(statistic=-2.129970166236807, pvalue=0.034916573358014256)

## Counting bombs

### 1)

In [ ]:
k = np.array([0,1,2,3,4,5], dtype = "int")
nk = np.array([229,211,93,35,7,1])
n_total = np.sum(nk)

lam = 537 / n_total # was given to us
print("Estimation of the parameter 𝜆 for Poisson distribution from our data: " + str(round(lam,7)))

distP = scipy.stats.poisson(lam)
expect = distP.pmf(np.arange(6))* 576

print ("Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5 (last cell is exactly 5):")
print(np.around(expect))

expect[5]=(1-distP.cdf(4))*576     

print ("Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5+ (last cell is 5+):")
print(np.around(expect))


Estimation of the parameter 𝜆 for Poisson distribution from our data: 0.9322917
Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5 (last cell is exactly 5):
[227. 211.  99.  31.   7.   1.]
Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5+ (last cell is 5+):
[227. 211.  99.  31.   7.   2.]


Last run result:

Estimation of the parameter 𝜆 for Poisson distribution from our data: 0.9322917

Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5 (last cell is exactly 5):

[227.  211.   99.   31.    7.    1.]

Expected number of regions 𝑛̃𝑘 with number of bombs from 0 to 5+ (last cell is 5+):

[227.  211.   99.   31.    7.    2.]

### 2)

In [ ]:
t_sum_calc = np.power(nk - expect, 2) / expect
bombs_t = np.sum(t_sum_calc)

print("Sample value of T: " + str(bombs_t))

Sample value of T: 1.169154675181436


Last run result:

Sample value of T: 1.169154675181436

### 3)

In [ ]:
t_p_value = 1 - scipy.stats.chi2.cdf(bombs_t, 4)

print("P-value: " + str(round(t_p_value,6)))
if (t_p_value < 0.05):
    print("Is low enough to reject the hypothesis about the Poisson distribution")
else:
    print("Is not low enough to reject the hypothesis about the Poisson distribution")
# sanity check
print("Sanity check: calculate with scipy.stats.chisquare:")
scipy.stats.chisquare([229,211,93,35,7,1],expect,1)

P-value: 0.883151
Is not low enough to reject the hypothesis about the Poisson distribution
Sanity check: calculate with scipy.stats.chisquare:


Power_divergenceResult(statistic=1.169154675181436, pvalue=0.8831505189191594)

Last run result:

P-value: 0.883151

Is not low enough to reject the hypothesis about the Poisson distribution.

Sanity check: calculate with scipy.stats.chisquare:

Power_divergenceResult(statistic=1.169154675181436, pvalue=0.8831505189191594)

### Russian cities, continued

In [ ]:
cities = np.genfromtxt('russian_cities_g_9k.csv')
cities_sig_level = 0.05

cities_a = np.min(cities)
cities_n = cities.size
cities_ln = np.log(cities)
cities_b = cities_n / (np.sum(cities_ln) - cities_n * np.log(cities_a))

dist = scipy.stats.pareto(cities_b, scale = cities_a)
ks_stat, ks_pvalue = scipy.stats.kstest(cities, dist.cdf)

print("KS P-value: " + str(ks_pvalue))

if (ks_pvalue < cities_sig_level):
    print("According to Kolmogorov-Smirnov method the data was not generated by the Pareto distributions")
else:
    print("According to Kolmogorov-Smirnov method the data was really generated by the Pareto distributions")

KS P-value: 2.0794160126346703e-12
According to Kolmogorov-Smirnov method the data was not generated by the Pareto distributions


Last run result:

KS P-value: 2.0794160126346703e-12

According to Kolmogorov-Smirnov method the data was not generated by the Pareto distributions